In [1]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
import re 
from tqdm.notebook import tqdm
import random

import shutil #이미지저장

import pandas as pd
import numpy as np
import os

import re #숫자만 남기기

In [2]:
#pip install selenium

In [3]:
#pip install webdriver-manager

오류 수정 내용 : 
1. 페이지 요청 오류 났을 시(너무 많은 요청 or 삭제된 계정) 모든 반환값을 -1로 저장
2. timestamp 안뜨는거 수정
3. 모든 반복문에 tqdm바(진행 상태 확인) 추가

In [2]:
def get_Posts_Followers_Followings(): #게시글 수, 팔로워 수, 팔로윙 수
    
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    #title = soup.find('title').text
    profile_countable_list = []
    
    try:
        #태그 위치
        li_tags = soup.select('.container ul > li')

        for li in li_tags:
            text = li.text.replace(' ','') #공백 지우기
            counts = int(re.sub('[^0-9]','',text)) #숫자만 냄구기
            profile_countable_list.append(counts)
        if len(profile_countable_list)<3:
            profile_countable_list = ['-','-','-']
    except:
        print('get_Posts_Followers_Followings 예외 발생!')
        profile_countable_list = ['-','-','-']  
    
    return profile_countable_list

In [3]:
def get_InfluId_Biography_ProfileUrl(): #id, biography, 프로필사진url
    
    
    #title = soup.find('title').text
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')   
    profile_uncountable_list = []
    
    try: 
        #container = soup.select_one('.container a')['href']
        container = soup.select_one('.container div.user__title > a').attrs['href']
        profile_uncountable_list.append(container[3:])


        biography = soup.select_one('.container .user__info-desc').text
        profile_uncountable_list.append(biography)


        user_img = soup.select_one('.container .user .user__img')
        user_img_s = str(user_img)
        left = user_img_s.find('url(')
        right = user_img_s.rfind(')')
        profile_uncountable_list.append(user_img_s[left+5:right-1])
        if len(profile_uncountable_list)<3:
            profile_uncountable_list = ['-','-','-']

    except:
        print('get_InfluId_Biography_ProfileUrl 예외 발생!')
        profile_uncountable_list = ['-','-','-']
        
    return profile_uncountable_list

In [4]:
def get_Followers_dict(url, followers):
    new_url = url + '/followers'    
    
    driver = webdriver.Chrome(executable_path=r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)
    #driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
    driver.get(new_url)
    
    #title = soup.find('title').text
    
    try:  
        time.sleep(0.5)
        count = []
        tmp = []
        f_var = 0
        followers_dict = {} #딕셔너리형태로 저장
        
        with tqdm(total=followers) as pbar:
            while True:
                
                if (len(count)-len(tmp)) > 100:
                    break

                source = driver.page_source
                soup = BeautifulSoup(source, 'html.parser')

                a_tag = soup.select('.container .content__list.followers.grid.infinite_scroll a')

                for a in a_tag:
                    id_url = a.attrs['href'] #https://dumpor.com/v/계정
                    followers_dict[id_url[21:]] = a.img['src'] # {계정이름 : 프로필이미지 url}

                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") #휠스크롤

                time.sleep(0.5)
                count.append('1')
                
                if f_var != len(followers_dict):
                    f_var = len(followers_dict)
                    tmp.append('1')
                    pbar.update(1)
                print('count:',len(count))
                print('tmp:  ',len(tmp))
                print('len(dict):',len(followers_dict))
                print('-------------------------------')
                
    except:
        return {'-':'-'}
    

    driver.close()
    return followers_dict

In [17]:
def time_change(x,nomalize = False):
    import datetime
    from datetime import date, datetime, timedelta
#     from datetime import timedelta,datetime
    from dateutil.relativedelta import relativedelta
    '''검색결과에 있는 문자를 날짜로 변경
    크롤링시 작성일자가 1분 전/1시간 전/1일 전으로 나타는 경우가 존재
    이러한 패턴을 일반적인 YY-MM-DD형태로 변환
    그리고 월별 계산이 중요하므로 모든 데이터는 1일로 통일
    
    Return
    -------
    x : datetime
        날짜형형태로 전환
        
    Example
    -------
    >>> x = "47분 전"
    >>> time_change(x)
    datetime.datetime(2020, 6, 1, 0, 0)
    
    >>> x2 = "2019.01.31"
    >>> time_change(x2)
    datetime.datetime(2019, 1, 1, 0, 0)
    '''
    
    number = re.compile('[0-9]+')
    today = date.today()#datetime.datetime.today().date()
    x = str(x)
    # 일자
    if " day" in x :
        d = re.sub(r'[^0-9]', '', x)#re.findall(number, x)[0]
        x = today - timedelta(days=int(d))
    elif " hour" in x:
        d = re.sub(r'[^0-9]', '', x)#re.findall(number, x)[0]
        x = today
    elif " minute" in x:
        d = re.sub(r'[^0-9]', '', x)#re.findall(number, x)[0]
        x = today
    #elif x == " month":
    #    x = today - timedelta(days=1)
    elif " month" in x: # '%Y년 %m월 %d일'형태
        m = re.sub(r'[^0-9]', '', x)
        x = today - relativedelta(months=int(m))
        #x = str(new_date.date())
    elif " year" in x: # '%Y년 %m월 %d일'형태
        y = re.sub(r'[^0-9]', '', x)
        x = today - relativedelta(years=int(y))
        #x = str(new_date.date())
    else :
        x = x
    
    #모든일자를 1일로 통일
    if nomalize:
        x = datetime.datetime.strptime(x.strftime('%Y-%m')+"-01","%Y-%m-%d")
    return x

In [6]:
def get_date():
    
    title = soup.find('title').text
    try:
        dates_text = soup.select('#card-page > .container .content__btns > div')[2].span.text
        return dates_text
    except:
        return -1

In [7]:
def get_post_url(posts):
    #url 수집
    url_halfs = []
    count = 0
    with tqdm(total=posts) as pbar:
        while True:     
            if count > posts:
                break

            source = driver.page_source
            soup = BeautifulSoup(source, 'html.parser')

            title = soup.find('title').text      
            try:
                #url저장
                url_source = soup.select('.container a')

                for url_half in url_source:
                    if '/c/' in str(url_half):
                        url_halfs.append(url_half.attrs['href'])           

                #중복제거
                url_halfs = list(set(url_halfs))
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") #휠스크롤
                time.sleep(0.5)
                count = count + 1
                pbar.update(1)
                #print(count)
            except:
                url_halfs.append('-1')
    print(f'수집된 url수:{len(url_halfs)}')
    return url_halfs

In [8]:
def get_post_text():
    main_text = soup.select_one('#card-page div.main__list').text
    return main_text

In [18]:
def get_likes_replies(): #[좋아요 개수, 댓글 수]
    likes_replies = []
    print("좋아요,댓글,날짜,다운로드",len(soup.select('#card-page > .container .content__btns > div')))
    try:
        likes_replies.append(soup.select('#card-page > .container .content__btns > div')[0].span.text)
        likes_replies.append(soup.select('#card-page > .container .content__btns > div')[1].span.text)
    except:
        print('get_likes_replies 예외발생')
        likes_replies.append(-1)
        likes_replies.append(-1)
    return likes_replies

In [19]:
def get_reply_info(replies): #댓글username : 댓글text
    replies_dict = {} #딕셔너리형태로 저장
    count = 0
    with tqdm(total=int(replies)) as pbar:
        while True:
            if (count > int(float(replies)*0.9)) or (len(replies_dict) > int(replies)): #걍 스크롤 50번이면 댓글 다 가져올거야 제발
                break

            source = driver.page_source
            soup = BeautifulSoup(source, 'html.parser')

            title = soup.find('title').text
            try:
                tag = soup.select('#card-page > .container .wrap.comments.mb-4 > div > div > div > div')
                replies_dict = {}
                text_list = []
                for a in tag:
                    n = 1
                    for b in a: #두개의 태그가 있어서..
                        n = n+1
                        text = b.text
                        if n%2==0: #id
                            text_list.append(text[1:])
                        else: #댓글
                            text_list.append(text)
            except:
                replies_dict = {'-':'-'}
                break
                

            
            replies_dict={text_list[i]: text_list[i + 1] for i in range(0, len(text_list), 2)}
            print("수집한 댓글 수 : ",len(replies_dict)) #확인해 몇개 받았는지
            
            pbar.update(1)
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") #휠스크롤

            time.sleep(1)
            count = count + 1
    
    return replies_dict

In [11]:
from selenium import webdriver

#add
### 15202623986    seulgistyles_
### 2043464728      style_mans
### 317091289        diegoosorio1
### 30316172873    joysstyles_
### 191403723        kostelevision
### 22566228417    k_fashion_looks
### 38980901318    dailyfashion_news
### 35641698457    irenebaefashion
### 1394688376      bsha_bsha
### 1297647769      coolguy9283
### 1996261596      style_88_

In [25]:
keyword = "style_mans" 
before_data= keyword + '_게시글.csv'
url = "https://dumpor.com/v/{}/".format(keyword)
os.makedirs('./{}'.format(keyword),exist_ok=True) #하위폴더 생성
os.makedirs('./{}/comms'.format(keyword),exist_ok=True) #하위폴더 생성
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

In [24]:
#driver options
options = Options()
options.add_argument('--start-maximized') #전체화면
options.add_argument(f'user-agent={headers}')
options.add_argument('--headless') #headless(화면안켜고)

In [27]:
#driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
driver = webdriver.Chrome(executable_path=r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)
driver.get(url) 


<ipython-input-27-3a0b7f061804>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)


In [15]:
temp = get_Posts_Followers_Followings()
temp1 = get_InfluId_Biography_ProfileUrl()
profile = {}

profile['user_name'] = temp1[0]    #계정
profile['biography'] = temp1[1]    #프로필
profile['media_count'] = temp[0]    #게시글수
profile['followed_by_count'] = temp[1] #팔로워수
profile['follow_count'] = temp[2]    #팔로윙수
profile['profile_pic_url'] = temp1[2] #프로필사진url

df = pd.DataFrame([profile])
df.to_csv('./{}/{}_profile.csv'.format(keyword,keyword), encoding='utf-8-sig')
df

,user_name,biography,media_count,followed_by_count,follow_count,profile_pic_url
0,style_mans,style mans magazin 😎,668,27946,5235,https://cdn2.dumpor.com/view?q=%3D%3DwM4YmZidT...


In [185]:
#시간 각오하고 돌려야 됨
temp2 = get_Followers_dict(url, temp[1])
follower = temp2

df1 = pd.DataFrame(follower.items(), columns=['follower_id', 'profile_img_url'])

<ipython-input-181-9d7afa5bfef2>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)


count: 1
tmp:   1
len(dict): 24
-------------------------------
count: 2
tmp:   2
len(dict): 48
-------------------------------
count: 3
tmp:   2
len(dict): 48
-------------------------------
count: 4
tmp:   2
len(dict): 48
-------------------------------
count: 5
tmp:   3
len(dict): 72
-------------------------------
count: 6
tmp:   3
len(dict): 72
-------------------------------
count: 7
tmp:   3
len(dict): 72
-------------------------------
count: 8
tmp:   4
len(dict): 96
-------------------------------
count: 9
tmp:   4
len(dict): 96
-------------------------------
count: 10
tmp:   4
len(dict): 96
-------------------------------
count: 11
tmp:   5
len(dict): 120
-------------------------------
count: 12
tmp:   5
len(dict): 120
-------------------------------
count: 13
tmp:   5
len(dict): 120
-------------------------------
count: 14
tmp:   5
len(dict): 120
-------------------------------
count: 15
tmp:   5
len(dict): 120
-------------------------------
count: 16
tmp:   5
len(dict):

count: 124
tmp:   31
len(dict): 721
-------------------------------
count: 125
tmp:   31
len(dict): 721
-------------------------------
count: 126
tmp:   31
len(dict): 721
-------------------------------
count: 127
tmp:   31
len(dict): 721
-------------------------------
count: 128
tmp:   31
len(dict): 721
-------------------------------
count: 129
tmp:   31
len(dict): 721
-------------------------------
count: 130
tmp:   31
len(dict): 721
-------------------------------
count: 131
tmp:   31
len(dict): 721
-------------------------------
count: 132
tmp:   31
len(dict): 721
-------------------------------



In [186]:
df1

,follower_id,profile_img_url
0,dizm007,https://cdn2.dumpor.com/view?q=%3D%3DANtcjYjNW...
1,borckrene.parada,https://cdn3.dumpor.com/view?q=2YjYygjM9QWaz91...
2,say_lovng_tme,https://cdn3.dumpor.com/view?q=%3D%3DgN2ImM4IT...
3,yongpari_,https://cdn3.dumpor.com/view?q=2YjYygjM9QWaz91...
4,jookkani,https://cdn3.dumpor.com/view?q=%3D%3DgN2ImM4IT...
...,...,...
716,jong_won_kim0912,https://cdn3.dumpor.com/view?q=%3D%3DgN2ImM4IT...
717,mirae_hng,https://cdn3.dumpor.com/view?q=2YjYygjM9QWaz91...
718,littleblue_messi,https://cdn1.dumpor.com/view?q=%3DYjNiJDOy0DZp...
719,yeeelim_0.0,https://cdn3.dumpor.com/view?q=%3D%3DgN2ImM4IT...


In [203]:
for row in tqdm(df1.iloc[:,0]):

    print(row)

dizm007
borckrene.parada
say_lovng_tme
yongpari_
jookkani
foot_ballector
05kim_pro14
p__jaewon
my.raculum
gkrmtlrm
jaesung123
ixthxmxd
ejunh23
jaehwan86
1been_37
piniyua12i77
city_in_seok
yeongryun91
kim_do_yoon1
pol_vengers_j
junopoly00
basement_freestyleclub
youra.0131
ikjunzzz
chefjun1121
jetty.0318
won2wonnie
minn._.seok
dkdlej88
valkyrie_x_rose
x_km.han_x
suriaan_
strawberry12244
jejuddakseawoo_paju
starmom_18
adidascokr
it_ji_yeon0130
cho94_28
gssj5030
chic_w.y
woosik__e
oct.enth
re4mo_myteamulsan
bogus.hoon
intaeee
khpark510
kimobcd
boombapgto
augustinus
xinjimaye
footballshirtthreads
go_d_jisoo
pedrosilva_stz
johee_joy
bagoggyeong146
gjfc__
kop._.chi
kimkimkimjiji
simkungdaddy
pog_js
best_fs__yongin
tomo.sports.club
mongjjjjjjj
76a_0b
jessicashih47
foriser_sport
channnn_ho
sseoyuvi
pumafootball_kr
l_want_b
s.minnns
gng_insta
wfc_bucheon
puma_rc17
coreaseller
d.woong
cr990221
dongzhe_in_japan
asiakiller_yzj
mok_ys
rollercost3
jungcm0414
kop_drm
sasa_footballshirts
wassiddassi
ra

In [231]:
f_biography = []
f_media_count = []
f_followed_by_count = []
f_follow_count = []

driver = webdriver.Chrome(executable_path=r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)
#driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
for row in tqdm(df1.iloc[:,0]):
#for row in tqdm(df1.iloc[185:,0]):
    print(row)
    temp_url = "https://dumpor.com/v/{}/".format(row)
    print(temp_url)
    driver.get(temp_url)
#     source = driver.page_source
#     soup = BeautifulSoup(source, 'html.parser')
    
    temp2_0 = get_Posts_Followers_Followings()
    print(temp2_0)
    print('-------------------------')
     
    temp2_1 = get_InfluId_Biography_ProfileUrl()
    print(temp2_1[1])
    
    f_biography.append(temp2_1[1])
    f_media_count.append(temp2_0[0])
    f_followed_by_count.append(temp2_0[1])
    f_follow_count.append(temp2_0[2])
df1['f_biography'] = f_biography
df1['f_media_count'] = f_media_count
df1['f_followed_by_count'] = f_followed_by_count
df1['f_follow_count'] = f_follow_count
df1.to_csv('./{}/{}_followers.csv'.format(keyword,keyword), encoding='utf-8-sig')
df1

<ipython-input-231-4ea599589649>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)


d__h_10
https://dumpor.com/v/d__h_10/
[196, 713, 167]
196   713   167
Do your best and god will do the rest먹스타그램,북스타그램,조카스타그램,레고스타그램 일상기록.
rb0734
https://dumpor.com/v/rb0734/
[0, 1, 560]
0   1   560

xaxo6.5
https://dumpor.com/v/xaxo6.5/
[7, 70, 112]
7   70   112

gnlw56688
https://dumpor.com/v/gnlw56688/
[0, 599, 3834]
0   599   3834
@MarineCorps354 <- 여기로 팔로우/좋아요 부탁드립니다 (본계정리중)
1_subin_bin
https://dumpor.com/v/1_subin_bin/
[30, 41, 487]
30   41   487
❤️본 계정은 인스타 정식 제휴계정 입니다❤️👉리스크 제로 하루2시간 투자🕜👉초보자가능👌 하루⌚ 40분만투자⌚👉핸드폰 Or PC 있을시 바로가능합니다💰수익보장100%수익 · 당일 수익 확인🕛👉주부&육아맘&투잡가능🔻오픈 톡 문의주세요🔻
songquelme
https://dumpor.com/v/songquelme/
[107, 831, 457]
107   831   457

ulsan_ifc
https://dumpor.com/v/ulsan_ifc/
[8, 148, 202]
8   148   202
⚽️울산 IFC⚽️⚽️유치부/초등부/성인레슨⚽️취민반/엘리트반/선수반/개인레슨⚽️초등부 상시테스트모집⚽️프로출신 지도자의 디테일한 지도력울산 4개지점 보유(옥동,삼산,굴화,구영)⬇️문의문의⬇️📮인스타DM📞010-3008-5626 📞052-261-0077
sonnyjersey
https://dumpor.com/v/sonnyjersey/
[114, 569, 877]
114   569   877
Sonheungmin Jersey Collection
cangeuni
https:

[425, 56, 311]
425   56   311
CAT is the BEST!Nice guys finish last!
j__m_222
https://dumpor.com/v/j__m_222/
[6, 51, 146]
6   51   146

htetnyeinoo
https://dumpor.com/v/htetnyeinoo/
[40, 71, 348]
40   71   348
𝑰 𝒂𝒎 🇲🇲
fc_return_
https://dumpor.com/v/fc_return_/
[2, 23, 31]
2   23   31
Fc리턴 20대로 이루어진 실력최하 풋살팀
bucketlittv
https://dumpor.com/v/bucketlittv/
[56, 591, 1392]
56   591   1392
형제가 함께 만드는 영상
gnas_gnoeyg_og
https://dumpor.com/v/gnas_gnoeyg_og/
[170, 447, 1162]
170   447   1162
'설계사' 이득 보다는'고객' 이득을 최우선으로.보험'점검'은 필수보험'준비'는 선택.보험점검&준비보험금청구&문의저축&연금문의건강검진&안과검진문의교통사고처리&합의문의배상책임처리&합의문의..무료상담문의010 9498 0480언제든지 연락주세요
wbcho16
https://dumpor.com/v/wbcho16/
[7, 126, 711]
7   126   711
찐맛집 소소한일상 공유합니다
chaeri_umpa
https://dumpor.com/v/chaeri_umpa/
[116, 147, 324]
116   147   324
#레사모 #레플러 #레플리카
art_director.gon
https://dumpor.com/v/art_director.gon/
[82, 146, 217]
82   146   217
art_director 🇰🇷korean. fashion. sneakers. soccer. Liverpool. Life. travel🇭🇰🇨🇳🇩🇪🇨🇿🇦🇹🏴󠁧󠁢󠁥󠁮󠁧󠁿🇪🇸🇫🇷
mount_jersey
https:/

[0, 28, 27]
0   28   27

15.103__26
https://dumpor.com/v/15.103__26/
[5, 0, 755]
5   0   755
오늘은 뭐 입지? 모카스우시 인스타 옷장 & 신발장 👟🩴🔄마켓 오픈전까지 비공개계정으로 닫아둘께요! 🙏🏻😭 #나이키 #디매 #나매 #사카이 #이지부스트350v2 #조던
bosstman1842
https://dumpor.com/v/bosstman1842/
[30, 189, 714]
30   189   714
라포르테
semin0417
https://dumpor.com/v/semin0417/
[901, 6080, 4393]
901   6080   4393
준석 준우아빠의 사생활 #이세민 #88#고선영 남편이라 써달라고함#이준석아빠 #이준우아빠#육아 #군인 #ROTC #118학군단#음악 #임창정#무조건맞팔 #소통#자전거 #라이딩 #로드 #MTB #스캇#스페셜라이즈드#매빅에어 #드론 #여행
octopuskorea
https://dumpor.com/v/octopuskorea/
[31951, 2708, 227]
31951   2708   227
Official Nameset and Patch Seller 100% 정품 오피셜 마킹,패치만을 판매하는 옥토퍼스사커입니다. 방문마킹은 12월 재개합니다. Phone : +821030716501
enb_square_football_academy
https://dumpor.com/v/enb_square_football_academy/
[36, 811, 5598]
36   811   5598
수원시 영통구 대학3로 7 나보나 스퀘어 4층경기대학교 야외 축구장
juhyeok112
https://dumpor.com/v/juhyeok112/
[4, 126, 102]
4   126   102
ㅈㅜㅎㅕㄱ
re._.member_yu
https://dumpor.com/v/re._.member_yu/
[11, 224, 532]
11   224   532
🥰 @hyeonzzin22 🥰


[8, 30, 632]
8   30   632

pamahwan
https://dumpor.com/v/pamahwan/
[339, 586, 1580]
339   586   1580

beckszone23
https://dumpor.com/v/beckszone23/
[0, 59, 263]
0   59   263

regular_min
https://dumpor.com/v/regular_min/
[116, 88, 239]
116   88   239
⚽️🎽🎮🎳🇰🇷✈🇯🇵🇹🇭🇻🇳🇷🇺
sung_ter09
https://dumpor.com/v/sung_ter09/
[1, 121, 468]
1   121   468
👫🏻 𝟸𝟶𝟷𝟿. 𝟷𝟸. 𝟷𝟸💍 𝟸𝟶𝟸𝟷. 𝟶𝟿. 𝟶𝟺💁🏻‍♂️ @sooooohyun53
ky__un
https://dumpor.com/v/ky__un/
[7, 445, 337]
7   445   337
🇰🇷
umbro_hyyuunn
https://dumpor.com/v/umbro_hyyuunn/
[400, 462, 509]
400   462   509
💎💎롯데김포공항점 엄브로💎💎☎️02)6116-2949롯데김포 엄브로 매니저.축구레플리카를 좋아하는 나의 계정.방문구매하시는 모든 고객님께 사은양말을 증정합니다.상품문의 디엠,방문,친구추가환영입니다😄😄#엄브로#umbro#롯데백화점#김포공항
h.tae_ynwa
https://dumpor.com/v/h.tae_ynwa/
[47, 1746, 989]
47   1746   989
@liverpoolfc
h._.junny_
https://dumpor.com/v/h._.junny_/
[549, 1043, 948]
549   1043   948
#REALMADRID/#대구fc/#포항스틸러스
js.k808
https://dumpor.com/v/js.k808/
[324, 1042, 3069]
324   1042   3069
Jun.k-선량한 제주도민🏝-前볼링선수🎳-ManchersterUTD-박지성때문에맨유를좋아함-제주도에서 박지성 박

[314, 1149, 1259]
314   1149   1259
주관적인 풋살화 소개 및 용품 소개 #풋살 #풋살화 #풋살화리뷰 #축구 #축구화 #축구화리뷰
ddoublek_d
https://dumpor.com/v/ddoublek_d/
[90, 551, 367]
90   551   367

chilgokha_service_area
https://dumpor.com/v/chilgokha_service_area/
[6, 40, 67]
6   40   67
#칠곡휴게소화장실 #칠곡휴게소부산방향 #칠곡휴게소자율식당 #칠곡휴게소 #대주산업 #대신기업 #한국도로공사
desigm_ihk
https://dumpor.com/v/desigm_ihk/
[158, 114, 104]
158   114   104
소소한 취미축구 유니폼 트레이닝복 컬렉션
topbuyshop_1004
https://dumpor.com/v/topbuyshop_1004/
[86, 251, 627]
86   251   627
❗해외 명품 99.9%최상(미러급)❗100% 실사 위주 업데이트❗피드에 없는 제품은 자유롭게 문의 주세요❗현금 결제👌카드결제👌👇모든 제품 문의는 아래 링크로👇DM🚫
jaemigoods_1000bro
https://dumpor.com/v/jaemigoods_1000bro/
[497, 763, 1006]
497   763   1006

sgraphics_fw
https://dumpor.com/v/sgraphics_fw/
[157, 713, 1563]
157   713   1563
에스그래픽스 풋볼웨어⚽️🇰🇷전사유니폼 할인 이벤트중🎁상의27000원 세트45000원🖥그래픽디자이너가 직접 만듭니다!👇카톡채널 “SGraphics”
chlms80
https://dumpor.com/v/chlms80/
[718, 124, 116]
718   124   116
"world wild east" 🇰🇷🇯🇵🇨🇳🇹🇭🇺🇸🇭🇰🇹

[30, 95, 54]
30   95   54
레사모 사수올로Serie A collection 🇰🇷🇮🇹SASSUOLO💚
gourgour88
https://dumpor.com/v/gourgour88/
[249, 774, 1147]
249   774   1147
본계정👉gourgour8Football Shirts Collection.
gourgour8
https://dumpor.com/v/gourgour8/
[1696, 1740, 3904]
1696   1740   3904
축스타그램 👉 gourgour88
zzangfc23kdh
https://dumpor.com/v/zzangfc23kdh/
[24, 415, 385]
24   415   385

traveler_leek
https://dumpor.com/v/traveler_leek/
[369, 1911, 520]
369   1911   520
여행쟁이#Juventus 🏳🏴#Traveler ✈️@doriso_hongcheon@jangselgi_fanstagram
ssamba_no.10
https://dumpor.com/v/ssamba_no.10/
[720, 1187, 1259]
720   1187   1259
8월 함께 달리실 분 DM 혹은 링크 참고 해주세요:)
moonlight_8mile_
https://dumpor.com/v/moonlight_8mile_/
[2300, 498, 1541]
2300   498   1541
Don't Look Back in Anger.⚾️DODGERS / NC DINOS / SOFTBANK HAWKS / ⚽️CHELSEA FC / FC BAYERN MÜNCHEN / JUVENTUS /🏈PATRIOTS / 🏀LAKERS / 📸NEX-5T
kkyutima_choi
https://dumpor.com/v/kkyutima_choi/
[110, 84, 94]
110   84   94
⚽️ 𝙵𝚘𝚘𝚝𝚋𝚊𝚕𝚕 & 𝚜𝚑𝚒𝚛𝚝✈ 𝙻𝚒𝚏𝚎 & 𝚃𝚛𝚒𝚙🇰🇷🇷🇺🇯🇵🇮🇹🇻🇦🇯🇵🇨🇳🇦🇪🇲🇻🇦🇺🇨🇳🇧🇷🇮🇹🇨🇵

[29, 407, 1846]
29   407   1846
영화 / 여행 / 맛집탐방 등등 일상
juneoh0330
https://dumpor.com/v/juneoh0330/
[28, 348, 1124]
28   348   1124

kyeong_yeon_kim
https://dumpor.com/v/kyeong_yeon_kim/
[1093, 963, 5952]
1093   963   5952
#fc서울 #fc서울수호신#한국축구뿌리부터튼튼히 #대한축구협회#카페 #맛집스타그램
duckjil_ismylife
https://dumpor.com/v/duckjil_ismylife/
[0, 1, 135]
0   1   135
대상만 바뀔뿐, 인생은 덕질의 연속미니멀리스트의 삶을 지향하는 맥시멀리스트
yk5327
https://dumpor.com/v/yk5327/
[0, 48, 202]
0   48   202

t9performancetraining
https://dumpor.com/v/t9performancetraining/
[665, 1140, 1308]
665   1140   1308
- 축구선수 전문 트레이닝센터 / 축구교실 / PT- 스트렝스 / 스피드 / 파워 / 밸런스 / 코어- 대한축구협회 체력강사 자격 (KFA)- 대한축구협회 풋살D강사 자격 (KFA)- 스포츠재활훈련전문가 (KSPO)Youtube:T9TV📺
positivceo
https://dumpor.com/v/positivceo/
[172, 2035, 1303]
172   2035   1303
💙 긍정디노의 하늘스타그램 💙🌤 @positivdino 👈 긍정라이프 브랜딩.💙🌤"인생은 하늘과도 같다."(흐린 날도 있겠지만, 반드시 맑은 날도 있다!).#긍정디노#하늘스타그램#일상하늘기록#1일1하늘 #행크#방돈인친#mkyu#열정대학생ㆍ🌿 (👇 클릭)
_son_h_s
https://dumpor.com/v/_son_h_s/
[44, 1055, 1048]
44   1055   1048
#일상 취미생활계정👉 @son_

,follower_id,profile_img_url,f_biography,f_media_count,f_followed_by_count,f_follow_count
0,dizm007,https://cdn2.dumpor.com/view?q=%3D%3DANtcjYjNW...,,0,1,27
1,borckrene.parada,https://cdn3.dumpor.com/view?q=2YjYygjM9QWaz91...,,1,150,1120
2,say_lovng_tme,https://cdn3.dumpor.com/view?q=%3D%3DgN2ImM4IT...,1991/9/19 💝🐶😜,121,1397,4774
3,yongpari_,https://cdn3.dumpor.com/view?q=2YjYygjM9QWaz91...,🇰🇷 ʟɪᴠᴇ ɪɴ #guangzhou 주재원’ ʟɪꜰᴇ(외노자)🇨🇳,259,987,2445
4,jookkani,https://cdn3.dumpor.com/view?q=%3D%3DgN2ImM4IT...,주깐,9,108,92
...,...,...,...,...,...,...
716,jong_won_kim0912,https://cdn3.dumpor.com/view?q=%3D%3DgN2ImM4IT...,견우 평택UTD NO.8 U12 윤우 평택UTD 3학년 NO.15,18,130,137
717,mirae_hng,https://cdn3.dumpor.com/view?q=2YjYygjM9QWaz91...,AZ MIRAE Branch '나는 성공하는중이다' 여의도 종합 금융서비스 우수인증...,362,497,361
718,littleblue_messi,https://cdn1.dumpor.com/view?q=%3DYjNiJDOy0DZp...,⚽️ 블루베.리메시 풋살일기 ⚽️⚽️ 왕초보 풋린이 🧏‍♀️ 🔥🔥⚽️......My...,42,191,199
719,yeeelim_0.0,https://cdn3.dumpor.com/view?q=%3D%3DgN2ImM4IT...,Bien,42,266,402


In [232]:
driver.close()

In [16]:
#driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
driver = webdriver.Chrome(executable_path=r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)
driver.get(url) 
temp3 = get_post_url(temp[0])

<ipython-input-16-e71efce85244>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)


  0%|          | 0/668 [00:00<?, ?it/s]

수집된 url수:668


In [21]:
url_ = {'url': temp3 }
df2 = pd.DataFrame(url_)
df2.to_csv('./{}/{}_게시글url.csv'.format(keyword,keyword), encoding='utf-8-sig')
df2

,url
0,/c/9773669645852929661
1,/c/6336282447139006831
2,/c/8464676982229353721
3,/c/1358205463673238931
4,/c/2341347826658309471
...,...
663,/c/8317514848745220841
664,/c/5124558692342706451
665,/c/9877304214009839171
666,/c/1521924596542121121


In [31]:
driver.close()

In [32]:
#게시글 정보(이것도 각오하고 돌려야됨)
import datetime
from datetime import date, datetime, timedelta

b_url = pd.read_csv('./{}/{}_게시글url.csv'.format(keyword,keyword))['url']
b_url = b_url.apply(lambda x : x.replace('https://dumpor.com',''))

url_s = pd.Series(temp3)
url_halfs_list = list(url_s[(url_s.apply(lambda x : any([url in x for url in b_url])))].values)

#드라이버 재시작
#driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)

#각 게시글에서 정보추출
nonetype = type(print()) #타입비교 용도
index_dict = {} #데이터 임시저장
df4_3 = pd.DataFrame()


for index, url_half in tqdm(enumerate(url_halfs_list)):
    #저장 초기화
    contents = {}
    #contents['shortcode'] = [] #계정은 위에서 받아놓았음 
    contents['timestamp'] = [] #
    contents['text'] = []
    contents['comment_count'] = []
    contents['liked_by_count'] = []
    contents['url'] = []
    real_url = f'https://dumpor.com{url_half}'
    
    driver.get(real_url)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    #사람처럼 행동 random하게 스크롤 작동
    ran_cnt = random.randint(1,2)
    ran_length = random.choice(np.arange(100,200,100))
    for count in range(0,ran_cnt):
        driver.execute_script(f"window.scrollTo(0, {ran_length})")
        ran_length *= 2
#         time.sleep(0.5)
    
    #dates
    try :
        date = get_date()
        timestamp = time_change(date)
    except :
        timestamp = da
        print('timestamp 예외발생')
    
    #본문
    try :
        text = get_post_text()
        #user_id = souop.select('a.sqdOP.yWX7d._8A5w5.ZIAjV')[0].text
    except :
        text = ''
    
    temp4 = get_likes_replies()
    #댓글수
    try :
        comment_count = temp4[1]
    except : 
        comment_count = []
    
    #좋아요수
    try : 
        liked_by_count = temp4[0]
    except:
        liked_by_count = []
        
    #최소화해놓으면 안되고 최소화 하지 않으면 작동안함()
#     random_stop = random.randint(1,2) #인간답게하려고 랜덤으로 2~4초 스탑
#     time.sleep(random_stop)
      
    #저장
    contents['timestamp'] = str(timestamp) #date
    contents['text'] = text
    contents['comment_count'] = comment_count
    contents['liked_by_count'] = liked_by_count
    contents['url'] = real_url
    
    url_title = url_half.replace('/','_')
    
    url_titles = []
    reply_dict = get_reply_info(temp4[1])
    for i in range(len(reply_dict)):
        url_titles.append(url_title[3:])
    
    df4_1 = pd.DataFrame({'comm_id': url_titles})
    df4_2 = pd.DataFrame(reply_dict.items(), columns = ['comm_by_username', 'comm_text'])
    df4 = pd.concat([df4_1,df4_2], axis = 1)
    df4_3 = pd.concat([df4_3, df4], axis = 0)

    #인덱스맞춰저장
    index_dict[index] = contents

#데이터프레임화

df3 = pd.DataFrame(index_dict)
df3 = df3.T

df4_3.to_csv('./{}/{}_댓글.csv'.format(keyword,keyword), encoding='utf-8-sig')
df3.to_csv('./{}/{}_게시글정보.csv'.format(keyword,keyword), encoding='utf-8-sig')
df3

<ipython-input-32-2555ec0560e3>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)


0it [00:00, ?it/s]

좋아요,댓글,날짜,다운로드 4


  0%|          | 0/5 [00:00<?, ?it/s]

수집한 댓글 수 :  1
수집한 댓글 수 :  5
수집한 댓글 수 :  5
수집한 댓글 수 :  5
수집한 댓글 수 :  5
좋아요,댓글,날짜,다운로드 4


...

  0%|          | 0/1 [00:00<?, ?it/s]

수집한 댓글 수 :  1
좋아요,댓글,날짜,다운로드 4


  0%|          | 0/6 [00:00<?, ?it/s]

수집한 댓글 수 :  1
수집한 댓글 수 :  1
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6


,timestamp,text,comment_count,liked_by_count,url
0,2018-09-09,스트릿 패션요즘 내 워너비 패션.#street#fashion#ootd#daily#d...,5,460,https://dumpor.com/c/9773669645852929661
1,2017-09-09,스트릿 패션커프스셔츠와 니트를 함께 매치한 코디#street #fashion #oo...,5,421,https://dumpor.com/c/6336282447139006831
2,2016-09-09,스트릿 패션스트릿패션에 있어서 트렌디함이 담겨있는 코디와 센스있는 아이템 선택!sw...,0,383,https://dumpor.com/c/8464676982229353721
3,2017-09-09,스트릿 패션톰브라운 트레이닝복 시밀러 룩#street #fashion #ootd #...,0,382,https://dumpor.com/c/1358205463673238931
4,2018-09-09,스트릿 패션슬슬 벚꽃놀이를 가볼까나~?🌸 -#해외 #스트릿패션 #웨크웨어 #패션 #...,0,249,https://dumpor.com/c/2341347826658309471
...,...,...,...,...,...
663,2017-09-09,스트릿 패션 시크하고 멋스러운 스카잔#street #fashion #ootd #da...,1,290,https://dumpor.com/c/8317514848745220841
664,2017-09-09,스트릿 패션#율리아나세르젠코 #UlyanaSergeenko패션에 관심있는 분이시라면...,3,267,https://dumpor.com/c/5124558692342706451
665,2018-09-09,스트릿 패션체크코트와 머스타드컬러비니는 사랑입니다.#해외 #스트릿패션 #웨크웨어 #...,4,459,https://dumpor.com/c/9877304214009839171
666,2016-09-09,"해외 스트릿패션펑키한 부츠와 미니멀한 룩.조형적인 이어링의 매치, 머리부터 발끝까지...",1,331,https://dumpor.com/c/1521924596542121121


In [29]:
df4.T

,0
d_seong22,화나네
paakwoo,아기자룡☺️☺️
y_one.me,헐 애기때랑 또가타….❤️❤️
siiiinn_2j,큩걸~❤️
30s.en,아나 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ귀엽냩ㅌㅌㅌㅌㅌㅌ
__mar_16,너는 어릴때부터 이뿼구나..
vorte_reve,누가봐도 아가론샤
yu.__.j_01,"어린 시절부터 완성형,,,베이비💕"


### 인플루언서
- 계정명     .. O
- 프로필     .. O
- 팔로워수    .. O
- 팔로윙수    .. O
- 비즈니스 계정 ..X
- 카테고리명   ..X
- 프로필사진url .. O
- 게시글 수    .. O

### 팔로워 리스트
- 계정명       ..O
- 프로필사진url ..O

### 포스트(게시글)
- 게시글 id .. X
- 날짜      .. O
- 타입      ..X
- 텍스트    .. O
- 댓글수    .. O
- 좋아요수  .. O
- 이미지캡션   ..X

### 좋아요_클릭
- 게시글id ..X
- 좋아요수  ..O
- 좋아요클릭id .. X

### 댓글
- 게시글id ..X
- 댓글id  .. O
- 댓글내용 .. O

### 팔로워 개인 프로필
- 계정명  .. O
- 팔로워수  .. O
- 팔로윙수  .. O
- 비즈니스 계정 ..X
- 카테고리명  .. X
- 프로필사진url.. O
- 게시글수  .. O

## ----------------------------------------여기까지--------------------------------------------------

In [437]:
url2 = 'https://dumpor.com/v/p_xoun/followers'

In [438]:
url3 = "https://dumpor.com/c/5350319975241489362"
driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
driver.get(url2)

<ipython-input-438-c7c79097c5e4>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)


In [321]:
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [447]:
a_tag = soup.select('.container .content__list.followers.grid.infinite_scroll a')
a_tag[0].img['src']

'https://cdn1.dumpor.com/view?q=%3DYjNiJDOy0DZpN3Xj52XmczQDJkMyEjN9U2bmgjMzUWO3UjN4I2M5MDNkhzY3MTOlhTM3czMhZWY3EGZ9g2bmQTL30jYjNmJBFUQBJ0aUNWLH9UQ90GZlZyXyQULslDWBlEO3N0azkGTYV1M9MGav91Yu9lJ0Vmbu4GZjJmZuEmbm5SMtUzayhmZu0WYydWY0Nnbp1Ddo91Yu91PnBnau42X3ADO2AjM1QTMxgDNyQzN1AjN281M5UjN2UzNzMTNwUDNwQzX3MzNxUzM0EjMvATNxgHM1EzcvkTMtUDO4IjLxUDdvY3L0Vmbu4GZjJmZuEmbm5SMtUzayhmZu0WYydWY0Nnbp9yL6MHc0RHa'

In [448]:
followers_dict={}
count = 0
while True:
    if(count > 1857):
        break

    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')

    a_tag = soup.select('.container .content__list.followers.grid.infinite_scroll a')

    for a in a_tag:
        id_url = a.attrs['href'] #https://dumpor.com/v/계정
        followers_dict[id_url[21:]] = a.img['src'] # {계정이름 : 프로필이미지 url}

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") #휠스크롤

    #followers_dict = set(followers_dict)
    time.sleep(0.5)
    count = count + 1


KeyboardInterrupt: 

In [372]:
tag = soup.select('#card-page > .container .wrap.comments.mb-4 > div > div > div > div')
follwers_dict = {}
id_list = []
text_list = []
for a in tag:

    user_id = a.p.text
    n = 1
    for b in a:
        n = n+1
        text = b.text
        if n%2==0: #id
            text_list.append(text[1:]) 
        else: #댓글
            text_list.append(text)
            
followers_dict={text_list[i]: text_list[i + 1] for i in range(0, len(text_list), 2)}
print(followers_dict)
        


{'muni_koon': '피드 잘보고가요! 자극받고갑니닷🔥소통해요!😊', 'chell_byeol': 'ㅎㅣ 몸매 머선129❤️', 'w_wpppi': '곰도리 두마리🐻🐻\u200d❄️', 'crescentreasures_sidra32': 'Collab? Dm @crescentreasures', '0529.gaeun.ig': '온니 넘 이쁘세요', 'luxeitem_02': '스타일 멋집니당~제 피드도 놀러오세용~^^', 'a_unxa': '인형이냐거', 'ye_e_jin': '아 예뿌다', 'y_one.me': '아 넘이뻐,,,,,,', '__mar_16': '너 혼혈이라며? 천국과 한국 ><', 'vegemmil': 'ㅂㄹ야', 'yu.__.j_01': '와…미췬…아이돌인줄💖', '20._june': '론아 갈 수록 넘 예쁘다 ❤️', 'kim_sj62': '헙 예쁘다', '8.26.seining': '오엠쥐~ 넌 비올때 우산쓰지마라,,,꽃에는 물이 필요하니깐,,,', 'cactus_ze': '헉 여신님 🥰🥰', '1.03lio': '마스크 써야지', 'auaaa.y': '왤케 예뿌냐 진짜', '30s.en': 'ㅈㄴ이뻐 존나 이뻐 존나이뻐 ㅈㄴㅇㅃㅇㅃㅇㅃ'}


In [221]:
#getCount_Posts_Followers_Followings()
#get_InfluId_Biography_ProfileUrl()
#get_Followers_dict(url, followers)


In [279]:
main_text = soup.select_one('#card-page div.main__list').text

In [280]:
main_text

'🌞ㅡ#teamfollowback #맞팔해요 #fllowme#셀스타그램#likeforlikes#얼스타그램 #선팔하면맞팔#전주카페#selca#selfie #팔로우100#뷰티#대전카페#첫줄#좋아요#셀스타그램#헬스#대전프로필#following#필라테스#ootd#청주#오오티디#맞팔#좋반#flf#f#lfl#첫줄반사#lfl#f4f'

In [282]:
dates_text = soup.select_one('#card-page div.main__user-desc').text
dates = dates_text[5+len('ja.lyong'):]
dates

'3 days ago'

'3 days ago'

datetime.date(2021, 8, 14)

In [ ]:
user_id_url

In [ ]:
user_id = user_id_url[3:]
user_id